### 合并数据集
pandas对象中数据的三种合并方法：
1. pandas.merge：实现的是数据库的连接操作
2. pandas.concat:沿着一条轴将多个对象堆叠起来
3. combine_first:可将重复数据拼接到一起，用一个对象的值填充另一个对象中的缺失值

#### 数据库风格的DataFrame合并
先来一个多对一的合并

In [6]:
import pandas as pd
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'], 'data2':range(3)})

In [7]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [8]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [9]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


这里没有指定用哪一列进行连接，merge方法会自动将重叠列的列名作为键，当然最好还是声明一下

In [11]:
pd.merge(df1,df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


如果列名不一样呢，就要分开指定一下：

In [12]:
df3 = pd.DataFrame({'key1':['b','b','a','c','a','a','b'],'data1':range(7)})

In [13]:
df3

,key1,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [14]:
df4 = pd.DataFrame({'key2':['a','b','d'], 'data2':range(3)})

In [15]:
df4

,key2,data2
0,a,0
1,b,1
2,d,2


In [16]:
pd.merge(df3,df4, left_on='key1', right_on='key2')

,key1,data1,key2,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


默认merge采用'inner'连接，还有'left'、'right'、'outer'

In [17]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


多对多合并

In [18]:
df5 = pd.DataFrame({'key':['a','b','a','b','d'], 'data':range(5)})

In [19]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [20]:
df5

,key,data
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [21]:
pd.merge(df1,df5,on='key', how='left')

,key,data1,data
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,a,5,0.0


多对多连接产生的是行的笛卡尔积，连接效果只影响结果中的键

In [22]:
pd.merge(df1,df5, how='inner')

,key,data1,data
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,6,1
5,b,6,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


多键合并

传入一个由列名组成的列表即可

In [23]:
left = pd.DataFrame({'key1':['foo', 'foo', 'bar'], 'key2':['one', 'two', 'one'], 'val':[1,2,3]})

In [24]:
right = pd.DataFrame({'key1':['foo','foo','bar','bar'], 'key2':['one','one','one','two'], 'val':[4,5,6,7]})

In [25]:
left

,key1,key2,val
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [26]:
right

,key1,key2,val
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [28]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,val_x,val_y
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


重复列名的处理

suffixes属性用于指定附加到左右两个DataFrame对象的重叠列名上的字符串

In [29]:
pd.merge(left, right, on='key1')

,key1,key2_x,val_x,key2_y,val_y
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [30]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,val_left,key2_right,val_right
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


merge函数的参数如下：
![]('images/7178691-35ca716a4f1b8475.png')
![]('images/7178691-c86672e733ceccd9.png')

根据索引合并

合并的键是它的索引时，传递left_index=True或right_index=True（或者两者都传）

In [14]:
left1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})

In [35]:
left1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [15]:
right1 = pd.DataFrame({'group_val':[3.5,7]}, index=['a','b'])

In [37]:
right1

,group_val
a,3.5
b,7.0


In [38]:
pd.merge(left1,right1, left_on='key', right_index=True)

,key,data1,group_val
0,b,0,7.0
1,b,1,7.0
6,b,6,7.0
2,a,2,3.5
4,a,4,3.5
5,a,5,3.5


使用外连接进行合并

In [39]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,data1,group_val
0,b,0,7.0
1,b,1,7.0
6,b,6,7.0
2,a,2,3.5
4,a,4,3.5
5,a,5,3.5
3,c,3,NaN


#### 多层索引数据

对于多层索引，必须指定用于合并键的多个列

In [2]:
import numpy as np
import pandas as pd

In [3]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio','Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002], 
                      'data': np.arange(5.)})

In [4]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'], [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [5]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [6]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [7]:
pd.merge(lefth, righth, right_index=True, left_on=['key1', 'key2'])

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


#### 使用join方法合并数据

join方法可以方便的实现索引合并，它还可用于合并多个带有相同或相似索引的DataFrame对象，但<span class="burk">要求没有重叠的列</span>

In [8]:
 left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                      index=['a', 'c', 'e'],
                      columns=['Ohio', 'Nevada'])

In [9]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [10]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [11]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [13]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


DataFrame的join方法默认使用的是左连接，保留左边表的行索引。它还支持在调用的DataFrame的列上，连接传递的DataFrame索引：

In [16]:
left1.join(right1, on='key')

,key,data1,group_val
0,b,0,7.0
1,b,1,7.0
2,a,2,3.5
3,c,3,NaN
4,a,4,3.5
5,a,5,3.5
6,b,6,7.0


对于简单的索引合并，你还可以向join传入一组DataFrame，下一节会介绍更为通用的concat函数，也能实现此功能：

In [17]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York','Oregon'])

In [19]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


#### 轴向连接（concat函数）

In [20]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [21]:
s1

a    0
b    1
dtype: int64

In [22]:
s2

c    2
d    3
e    4
dtype: int64

In [23]:
s3

f    5
g    6
dtype: int64

In [25]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认情况下，concat是在axis=0上工作的，最终产生一个新的Series。如果传入axis=1，则结果就会变成一个DataFrame（axis=1是列）：

In [27]:
pd.concat([s1,s2,s3], axis=1, sort=True)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [28]:
s4 = pd.concat([s1, s3])

In [29]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [31]:
pd.concat([s1,s4],axis=1, sort=True)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [34]:
pd.concat([s1,s4],join='inner',axis=1)

,0,1
a,0,0
b,1,1


在这个例子中，f和g标签消失了，是因为使用的是join='inner'选项。

你可以通过join_axes指定要在其它轴上使用的索引：

In [35]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


在连接轴上创建一个层次化索引。使用keys参数即可达到这个目的：

In [36]:
result = pd.concat([s1, s1, s3], keys=['one','two', 'three'])

In [37]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [38]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


如果沿着axis=1对Series进行合并，则keys就会成为DataFrame的列头：

In [40]:
pd.concat([s1, s2, s3], axis=1, keys=['one','two', 'three'], sort=True)

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


同样的逻辑也适用于DataFrame对象：

In [41]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],columns=['three', 'four'])

In [42]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [43]:
df2

,three,four
a,5,6
c,7,8


In [46]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],sort=True)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

如果传入的不是列表而是一个字典，则字典的键就会被当做keys选项的值：

In [47]:
pd.concat({'level1':df1, 'level2':df2}, axis=1, sort=True)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

最后一个关于DataFrame的问题是，DataFrame的行索引不包含任何相关数据：

In [48]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [49]:
df1

,a,b,c,d
0,-0.738007,-0.366333,-0.166875,0.725260
1,0.514736,0.649835,-0.911457,0.434485
2,0.117563,0.671846,0.372962,-1.208758


In [50]:
df2

,b,d,a
0,-2.246495,-0.402549,-0.030828
1,0.579256,0.545820,-1.543652


In [55]:
pd.concat([df1, df2], ignore_index=True, sort=True)

,a,b,c,d
0,-0.738007,-0.366333,-0.166875,0.725260
1,0.514736,0.649835,-0.911457,0.434485
2,0.117563,0.671846,0.372962,-1.208758
3,-0.030828,-2.246495,NaN,-0.402549
4,-1.543652,0.579256,NaN,0.545820


concat方法的各种参数参考官方文档：
![](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html?highlight=concat#pandas.concat)

#### 用传递对象中的数据为调用对象补齐缺失数据

In [56]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [57]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [59]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [60]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN
